In [16]:
import random
from lang import load_data_int_seq
from utils import eq_encoder, is_eq_valid,normalize_0_1
from typing import List
from models.rl.env import IntegerSequenceEnv, get_current_position, encode_with_lang, decode_with_lang, eq_to_seq
import gym, ray
from ray.rllib.agents import ppo
from ray.rllib.agents import dqn
import numpy as np

%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
ray.init(ignore_reinit_error=True)


2021-07-14 12:23:59,402	INFO worker.py:745 -- Calling ray.init() again after it has already been called.


In [18]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:1]

In [19]:
def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:

    if len(output_sequence) != len(target_sequence):
        raise AssertionError("sequence size don't match: " + ','.join(str(e) for e in output_sequence)
 + " | " + ','.join(str(e) for e in target_sequence))

    magnitude: float = 0.0

    combined_seq = np.vstack([output_sequence, target_sequence]) 
    norm_comb_seq = normalize_0_1(combined_seq)

    norm_output_seq = norm_comb_seq[0]
    norm_target_seq = norm_comb_seq[1]

    
    for x, y in zip(norm_target_seq, norm_output_seq):
        magnitude += abs(x - y)#**2

    # magnitude /= len(norm_target_seq)

    return 10 - (magnitude * 100)



In [20]:
compare_sequences([1,2, 3, 100], [1,2,3, 100])

10.0

In [21]:
def evaluate_candidate_eq(candidate: str, int_seq: List[int]) -> float:
    if is_eq_valid(candidate) == False:
        return -MAX_PENALTY_MAGNITUDE

    output_sequence = eq_to_seq(candidate, 9)

    if np.count_nonzero(output_sequence) < 1:
        return -MAX_PENALTY_MAGNITUDE

    return compare_sequences(output_sequence, int_seq)

In [22]:
env = IntegerSequenceEnv({"data": train, "output_length": 9, "input_lang": input_lang, "output_lang": output_lang, "evaluate": evaluate_candidate_eq})

In [23]:
env.step(5)

(([7, -1, -1, -1, -1, -1, -1, -1, -1], [2, 3, 4, 5, 6, 7, 8, 9, 10]),
 0,
 False,
 {})

In [24]:
env.step(1)

(([7, 3, -1, -1, -1, -1, -1, -1, -1], [2, 3, 4, 5, 6, 7, 8, 9, 10]),
 0,
 False,
 {})

In [25]:
env.reset()

([-1, -1, -1, -1, -1, -1, -1, -1, -1], [2, 3, 4, 5, 6, 7, 8, 9, 10])

In [26]:
# trainer = ppo.PPOTrainer(env=IntegerSequenceEnv, config={
#     "env_config": {
#         "data": train,
#         "output_length": 9,
#         "input_lang": input_lang,
#         "output_lang": output_lang,
#         "evaluate": evaluate_candidate_eq
#     },
#     "num_envs_per_worker": 1,
#     # "train_batch_size": 10000,
#     "num_workers": 1,
#     "framework": "torch"
# })

2021-07-14 12:24:14,497	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [27]:
trainer = dqn.DQNTrainer(env=IntegerSequenceEnv, config={
    "env_config": {
        "data": train,
        "output_length": 9,
        "input_lang": input_lang,
        "output_lang": output_lang,
        "evaluate": evaluate_candidate_eq
    },
    "num_envs_per_worker": 1,
    # "train_batch_size": 10000,
    "num_workers": 1,
    "framework": "torch"
})

2021-07-14 12:24:23,463	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [31]:

for i in range(100):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(result["episode_reward_mean"])
#    print(result)

   if i % 100 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint, "i: ", i)

StopIteration: 

In [94]:
result = trainer.train()

In [ ]:
result["episode_reward_max"]